# Sample Notebook
Showcases the different steps taken in the Metaflow Flow

In [1]:
# Extension Reloads
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import re 
import sys
sys.path.append("../src")
import preprocess as pre
import utils as u

import tensorflow as tf
from sklearn.model_selection import train_test_split

### Process Data

In [3]:
#data = pd.read_csv("../data/complaints.csv")
data = pd.read_csv("../data/consumer_complaints_with_narrative.csv")
#data = pd.read_csv("../data/archive/consumer_complaints.csv")

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, data['consumer_disputed'].values, 
                                                                                test_size=0.2, random_state=42)

In [32]:
X_train.head()

,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company,state,zip_code,company_response,timely_response,consumer_disputed
11710,Credit card,NaN,Identity theft / Fraud / Embezzlement,NaN,I received a call from this phone # saying tha...,Capital One,MS,392XX,Closed with explanation,Yes,0
10230,Credit card,NaN,Other,NaN,I started using this service on XXXX as i have...,"PayPal Holdings, Inc.",OH,440XX,Closed with explanation,Yes,0
19574,Debt collection,I do not know,Disclosure verification of debt,Right to dispute notice not received,Hi i am trying to get some help in reaching ou...,Encore Capital Group,CA,951XX,Closed with explanation,Yes,0
23011,Prepaid card,General purpose card,Fraud or scam,NaN,I 've been a client of Rush card for 5 years a...,"Empowerment Ventures, LLC",MD,207XX,Closed with non-monetary relief,Yes,0
44707,Debt collection,I do not know,Communication tactics,Called after sent written cease of comm,Enhanced recovery systems called my job XXXX w...,ERC,NY,100XX,Closed with non-monetary relief,Yes,0


In [27]:
data.iloc[X_train].shape

ValueError: invalid literal for int() with base 10: 'Credit card'

In [4]:
numeric_features_df = data.copy()
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].str.replace('X', '0', regex=True)
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].str.replace(r'\[|\*|\+|\-|`|\.|\ |\$|\/|!|\(', '0', regex=True)
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].fillna(0)
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].astype('int32')
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].apply(lambda x: x//10000)

In [11]:
one_hot_features = ['product', 'sub_product', 'company_response', 'state', 'issue']
ohf_df = data[one_hot_features].copy()

for feature in one_hot_features:
    ohf_df[feature] = ohf_df[feature].astype("category").cat.codes

In [6]:
text_features = ['consumer_complaint_narrative']
text_features_df = data[text_features].copy()

In [12]:
print(numeric_features_df.shape)
print(ohf_df.shape)
print(text_features_df.shape)

(66799, 11)
(66799, 5)
(66799, 1)


In [13]:
final_df = pd.concat([text_features_df,ohf_df,numeric_features_df['zip_code']],axis=1)

In [14]:
final_df.shape

(66799, 7)

In [15]:
data.shape

(66799, 11)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
#data = data.rename(columns = {'zipcode':'zip_code'})

In [4]:
# data = data[data['consumer_complaint_narrative'].notnull()]
# data = data.sample(frac=1, replace=False).reset_index(drop=True)
# data = data.replace(r'\s', ' ', regex=True)
# data.to_csv('../data/input_data.csv', index=False)

In [8]:
# data.columns = pre.change_column_names(data.columns)
# data_pre = pre.column_selection(data)

In [9]:
# data_pre['c'] = data_pre['consumer_disputed'].map({'Yes': 1, 'No': 0})
# data_pre = data_pre.drop('consumer_disputed', axis=1)
# data_pre = data_pre.rename(columns={"c": "consumer_disputed"})

In [10]:
# data_pre['consumer_disputed'].value_counts()

0    51229
1    15577
Name: consumer_disputed, dtype: int64

**Process Numeric Features**

In [3]:
numeric_features_df = data.copy()
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].str.replace('X', '0', regex=True)
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].str.replace(r'\[|\*|\+|\-|`|\.|\ |\$|\/|!|\(', '0', regex=True)
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].fillna(0)
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].astype('int32')
numeric_features_df['zip_code'] = numeric_features_df['zip_code'].apply(lambda x: x//10000)

# import re
# numeric_features = ['zip_code']
# numeric_features_df = data_pre[numeric_features].copy()
# for i in numeric_features:
#     numeric_features_df[i] = numeric_features_df[i].apply(lambda x: x.replace('X', '0') if x==x else '0')
#     numeric_features_df[i] = numeric_features_df[i].apply(lambda x: x.replace(r'\[|\*|\+|\-|`|\.|\ |\$|\/|!|\(', '0'))
#     numeric_features_df[i] = numeric_features_df[i].apply(lambda x: re.sub('[^a-zA-Z0-9]','', x))
#     numeric_features_df[i] = numeric_features_df[i].fillna(0)
#     numeric_features_df[i] = numeric_features_df[i].astype('int32')
#     numeric_features_df[i] = numeric_features_df[i].apply(lambda x: x//10000)
    
numeric_x = [numeric_features_df['zip_code'].values]

**One Hot Features**

In [4]:
one_hot_features = ['product', 'sub_product', 'company_response', 'state', 'issue']
ohf_df = data.copy()

for feature in one_hot_features:
    ohf_df[feature] = ohf_df[feature].astype("category").cat.codes
    
one_hot_x = [np.asarray(tf.keras.utils.to_categorical(ohf_df[feature_name].values)) for feature_name in one_hot_features]

In [5]:
for col in one_hot_features:
    print(col)
    print(data[col].nunique())

product
11
sub_product
45
company_response
5
state
60
issue
90


**Text Features**

In [6]:
text_features = ['consumer_complaint_narrative']
text_features_df = data[text_features].copy()
embedding_x = [np.asarray(text_features_df[feature_name].values).reshape(-1) for feature_name in text_features]

**Training Data**

In [ ]:
# embedding_x = [np.asarray(text_features_df[feature_name].values).reshape(-1) for feature_name in text_features]
# one_hot_x = [np.asarray(tf.keras.utils.to_categorical(ohf_df[feature_name].values)) for feature_name in one_hot_features]
# numeric_x = [numeric_features_df['zip_code'].values]

In [8]:
X = one_hot_x + numeric_x + embedding_x
y = np.asarray(data["consumer_disputed"], dtype=np.uint8).reshape(-1)

### Create Model

In [18]:
import tensorflow_hub as hub

def get_model(show_summary=True):
    """
    Function defines a Keras model and returns the model as Keras object
    Args:
        num_products = 12
        num_sub_products = 47
        num_company_responses = 5
        num_states = 66
        num_issues = 91
        show_summary
    """    
    # one-hot categorical features
    num_products = 11
    num_sub_products = 45
    num_company_responses = 5
    num_states = 60
    num_issues = 90

    input_product = tf.keras.Input(shape=(num_products,), name="product_xf")
    input_sub_product = tf.keras.Input(shape=(num_sub_products,), name="sub_product_xf")
    input_company_response = tf.keras.Input(shape=(num_company_responses,), name="company_response_xf")
    input_state = tf.keras.Input(shape=(num_states,), name="state_xf")
    input_issue = tf.keras.Input(shape=(num_issues,), name="issue_xf")
    
    # numeric features
    input_zip_code = tf.keras.Input(shape=(1,), name="zip_code_xf")

    # text features
    input_narrative = tf.keras.Input(shape=(1,), name="narrative_xf", dtype=tf.string)

    # embed text features
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    embed = hub.KerasLayer(module_url)
    reshaped_narrative = tf.reshape(input_narrative, [-1])
    embed_narrative = embed(reshaped_narrative) 
    deep_ff = tf.keras.layers.Reshape((512, ), input_shape=(1, 512))(embed_narrative)
    
    deep = tf.keras.layers.Dense(256, activation='relu')(deep_ff)
    deep = tf.keras.layers.Dense(64, activation='relu')(deep)
    deep = tf.keras.layers.Dense(16, activation='relu')(deep)

    wide_ff = tf.keras.layers.concatenate([input_product, input_sub_product, 
                                           input_company_response, 
                                           input_state, input_issue, 
                                           input_zip_code])
    
    wide = tf.keras.layers.Dense(16, activation='relu')(wide_ff)


    both = tf.keras.layers.concatenate([deep, wide])

    output = tf.keras.layers.Dense(1, activation='sigmoid')(both) 

    _inputs = [input_product, input_sub_product, input_company_response,  
               input_state, input_issue, input_zip_code, input_narrative]

    keras_model = tf.keras.models.Model(_inputs, output)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    #optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    keras_model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',  
                        metrics=[tf.keras.metrics.BinaryAccuracy(),
                                 tf.keras.metrics.TruePositives(),
                                 tf.keras.metrics.Precision(),
                                 tf.keras.metrics.AUC(),
                                 tf.keras.metrics.Accuracy(),
                                 tf.keras.metrics.Recall()])
    if show_summary:
        keras_model.summary()

    return keras_model

In [19]:
model = get_model(show_summary=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
narrative_xf (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
tf.reshape (TFOpLambda)         (None,)              0           narrative_xf[0][0]               
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 512)          256797824   tf.reshape[0][0]                 
__________________________________________________________________________________________________
reshape (Reshape)               (None, 512)          0           keras_layer[0][0]                
______________________________________________________________________________________________

In [21]:
stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))

In [22]:
stringlist

['Model: "model"',
 '__________________________________________________________________________________________________',
 'Layer (type)                    Output Shape         Param #     Connected to                     ',
 '==================================================================================================',
 'narrative_xf (InputLayer)       [(None, 1)]          0                                            ',
 '__________________________________________________________________________________________________',
 'tf.reshape (TFOpLambda)         (None,)              0           narrative_xf[0][0]               ',
 '__________________________________________________________________________________________________',
 'keras_layer (KerasLayer)        (None, 512)          256797824   tf.reshape[0][0]                 ',
 '__________________________________________________________________________________________________',
 'reshape (Reshape)               (None, 512)         

In [25]:
short_model_summary = " \n".join(stringlist)
print(short_model_summary)

Model: "model" 
__________________________________________________________________________________________________ 
Layer (type)                    Output Shape         Param #     Connected to                      
narrative_xf (InputLayer)       [(None, 1)]          0                                             
__________________________________________________________________________________________________ 
tf.reshape (TFOpLambda)         (None,)              0           narrative_xf[0][0]                
__________________________________________________________________________________________________ 
keras_layer (KerasLayer)        (None, 512)          256797824   tf.reshape[0][0]                  
__________________________________________________________________________________________________ 
reshape (Reshape)               (None, 512)          0           keras_layer[0][0]                 
____________________________________________________________________________________

In [14]:
model.fit(x=X, y=y,batch_size = 32,validation_split=0.2, epochs=5)

Epoch 1/5
1670/1670 [==============================] - 154s 89ms/step - loss: 0.5296 - binary_accuracy: 0.7661 - true_positives_1: 16.6074 - precision_1: 0.2983 - accuracy: 0.0000e+00 - recall_1: 0.0017 - val_loss: 0.5071 - val_binary_accuracy: 0.7746 - val_true_positives_1: 71.0000 - val_precision_1: 0.6961 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0233
Epoch 2/5
1670/1670 [==============================] - 149s 89ms/step - loss: 0.5078 - binary_accuracy: 0.7708 - true_positives_1: 226.4817 - precision_1: 0.5869 - accuracy: 0.0000e+00 - recall_1: 0.0347 - val_loss: 0.5075 - val_binary_accuracy: 0.7741 - val_true_positives_1: 96.0000 - val_precision_1: 0.6038 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0315
Epoch 3/5
1670/1670 [==============================] - 147s 88ms/step - loss: 0.5063 - binary_accuracy: 0.7661 - true_positives_1: 336.3489 - precision_1: 0.5909 - accuracy: 0.0000e+00 - recall_1: 0.0541 - val_loss: 0.5080 - val_binary_accuracy: 0.7759 - val_true_positives_1: 11

In [21]:
model.evaluate(X, y)

2088/2088 [==============================] - 70s 34ms/step - loss: 0.5469 - binary_accuracy: 0.7668 - true_positives: 0.0000e+00 - precision: 0.0000e+00 - accuracy: 0.0000e+00 - recall: 0.0000e+00


[0.5468790531158447, 0.7668322920799255, 0.0, 0.0, 0.0, 0.0]

In [22]:
y_proba = model.predict(X)
y_proba.round(2)

array([[0.33],
       [0.29],
       [0.33],
       ...,
       [0.2 ],
       [0.27],
       [0.24]], dtype=float32)

In [37]:
for index,value in enumerate(y_proba):
    if value[0]>0.5:
        print(index,value)
        break

In [28]:
y_proba[760][0]

0.32979256

In [30]:
y[0]

1